# Поиск похожих каналов

Основной принцип:
1. Берем целевой канал, собираем в нем ники комментаторов.
2. Смотрим, комментирует ли комментатор от лица канала либо есть ли у него в описании ссылка на канал.
3. Если да, то добавляем этот канал к списку целевых каналов на сканирование.

In [22]:
import load_env

In [23]:
import asyncio
import logging
import re

import pyrogram
from fsspec.implementations.local import LocalFileSystem
from tqdm.auto import tqdm

from chat_cache import ChatCacheItem
from progress import ProgressKeeper
from scanner import Scanner
from utils import ensure_ats, get_nicknames

logging.getLogger("pyrogram").setLevel("ERROR")
logging.getLogger("urllib3").setLevel("ERROR")

In [24]:
LIMIT_HISTORY = 200  # насколько лезть вглубь чата
LIMIT_DISCUSSION = 30  # насколько лезть вглубь ветки комментариев
LIMIT_BATCH = 1000  # сколько сообщений за присест обрабатывать

MIN_SUBSCRIBERS = 500  # сколько должно быть подписчиков, чтобы сканировать посты канала
TARGET_WORDS = ["инвест"]  # какие слова ищем в названии или в био, чтобы сканировать канал

In [25]:
fs = LocalFileSystem()
scanner = Scanner(
    [
        "79852227949", 
        "79934962253", 
        "79037895690",
        "79934957590",
    ], 
    fs)
progress = ProgressKeeper(fs)

In [26]:
async def extract_from_bio(chat_id: str) -> tuple[set[str], set[str]]:
    try:
        chat = await scanner.get_chat(chat_id)
    except pyrogram.errors.PeerIdInvalid:
        return set(), set()

    nicknames = get_nicknames(
        " ".join([el for el in [chat.bio, chat.description] if el])
    )
    return await tell_channels_from_users(nicknames)


async def tell_channels_from_users(nicknames: set[str]):
    channels, users = set(), set()
    for nickname in nicknames:
        try:
            chat = await scanner.get_chat(nickname)
        except (pyrogram.errors.UsernameNotOccupied, pyrogram.errors.UsernameInvalid):
            continue

        if chat.type == pyrogram.enums.ChatType.CHANNEL:
            channels |= {nickname}
        elif chat.type == pyrogram.enums.ChatType.PRIVATE:
            users |= {nickname}

    return ensure_ats(channels), ensure_ats(users)


async def extract_from_posts(chat_id: str, pbar: tqdm) -> tuple[set[str], set[str]]:
    channels, users = set(), set()
    async for msg in scanner.get_chat_history(chat_id, LIMIT_HISTORY):
        extracted_channels, extracted_users = await extract_from_discussions(
            chat_id, msg.id, pbar
        )
        channels |= extracted_channels
        users |= extracted_users

        nicknames = get_nicknames(msg.text)
        extracted_channels, extracted_users = await tell_channels_from_users(nicknames)
        channels |= extracted_channels
        users |= extracted_users

    return ensure_ats(channels), ensure_ats(users)


async def extract_from_discussions(chat_id: str, msg_id: int, pbar: tqdm):
    channels, users = set(), set()

    async for reply in scanner.get_discussion_replies(
        chat_id, msg_id, LIMIT_DISCUSSION
    ):
        pbar.update()

        reply: pyrogram.types.Message

        if (
            reply.sender_chat
            and reply.sender_chat.type == pyrogram.enums.ChatType.CHANNEL
        ):
            channels |= {reply.sender_chat.username}
            scanner.chat_cache[reply.sender_chat.username] = ChatCacheItem(
                reply.sender_chat
            )

        elif reply.from_user:
            users |= {reply.from_user.username or reply.from_user.id}

            # внутри объекта from_user: User нет bio, поэтому нет смысла хранить его в кэше
            # scanner.chat_cache[reply.from_user.username] = ChatCacheItem(
            # reply.from_user
            # )

    return ensure_ats(channels), ensure_ats(users)


async def channel_eligible(chat_id: str) -> bool:
    chat: pyrogram.types.Chat = await scanner.get_chat(chat_id)
    count = await scanner.get_chat_members_count(chat_id)

    found = False
    for word in TARGET_WORDS:
        compiled = re.compile(word, re.IGNORECASE)
        found = found or bool(chat.title and re.findall(compiled, chat.title))
        found = found or bool(
            chat.description and re.findall(compiled, chat.description)
        )

    return count > MIN_SUBSCRIBERS and found


In [27]:

pbar = tqdm(total=LIMIT_BATCH)

async with scanner.session(pbar), progress.session(pbar):
    while pbar.last_print_n < LIMIT_BATCH:
        if progress.new_users:
            with progress.pop_user() as user_to_scan:
                pbar.set_postfix({"user": user_to_scan})
                progress.schedule(*(await extract_from_bio(user_to_scan)))

        elif progress.new_channels:
            with progress.pop_channel() as channel_to_scan:
                if (
                    channel_to_scan not in progress.scanned_channels
                    and await channel_eligible(channel_to_scan)
                ):
                    pbar.set_postfix({"channel": channel_to_scan})

                    progress.schedule(*(await extract_from_bio(channel_to_scan)))
                    progress.schedule(
                        *(await extract_from_posts(channel_to_scan, pbar))
                    )
        else:
            break

        pbar.update()


  0%|          | 0/1000 [00:00<?, ?it/s]

In [28]:
chats = []

async with scanner.session():
    pbar = tqdm(progress.scanned_channels | progress.new_channels)
    for chat_id in pbar:
        pbar.set_postfix_str(chat_id)
        try:
            chat: pyrogram.types.Chat = await scanner.get_chat(chat_id)
            members_count = await scanner.get_chat_members_count(chat_id)
            eligible = await channel_eligible(chat_id)
        except RuntimeError:
            continue
        if eligible:
            chats.append(
                (
                    chat.username,
                    chat.title,
                    chat.description,
                    members_count,
                )
            )

import pandas as pd
pd.DataFrame(chats, columns=['username', 'title', 'description', 'members_count']).sort_values("members_count", ascending=False)


  0%|          | 0/199 [00:00<?, ?it/s]

,username,title,description,members_count
5,harmfulinvestor,Вредный Инвестор,None,18513
1,atsogoev,Artem Tsogoev,💰 Канал Артёма Цогоева о недвижимости и инвест...,10174
14,trade_talk_tech,Trade Talk,Канал и подкаст о долгосрочных инвестициях.\n\...,7059
11,truevalue,Truevalue,Виктор Тунёв. Избранное о макроэкономике и MMT...,6619
8,sf_education,Курилка в Голдмане | SF Education,"Обучение по темам: финансы, инвестиции, работа...",6570
13,simpleestate,SimpleEstate | Инвестиции,Инвестиционная платформа коммерческой недвижим...,4071
4,unsaid_and_dirty,"Грязь, долги и инвестиции",None,2883
12,iRea1ty,Новостройки/недвижимость/инвестиции,"Помогу купить жильё для себя, аренды, перепрод...",2735
15,flipping_invest,Инвестиции в редевелопмент | Алексей Лещенко,Проекты с доходностью от 50% в год в недвижимо...,2083
0,d_smirnovv,Смирнов Дмитрий | Опыт инвестора,Авторский блог частного инвестора. Только личн...,1662
